# Retail Store Analaysis

In [1]:
import pandas as pd
data = pd.read_excel("Online Retail.xlsx")

*Dataste Overview*

In [2]:
# The total rows in the Dataset
data.shape[0]

541909

In [3]:
# The column in the Dataset
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Cleaning and Preprocessing 

*Removing Duplicate Values*

In [6]:
data.duplicated().sum()

np.int64(5268)

In [7]:
data = data.drop_duplicates()
data.duplicated().sum()

np.int64(0)

In [8]:
data.shape[0]

536641

*Handling Null Values*

In [9]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

In [10]:
data['Description'] = data['Description'].fillna(value = 'No Description')
# Removing rows with null values in CustomerId column
data = data.dropna(subset=['CustomerID'])
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [11]:
data.shape[0]

401604

*Data Type Conversion*

In [12]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [13]:
data['CustomerID'] = data['CustomerID'].astype(int)
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

## Data Aggregation

In [14]:
# Total Quantity Sold per Product (StockCode)
product_sales = data.groupby('StockCode')['Quantity'].sum().reset_index()
product_sales.head(5)

,StockCode,Quantity
0,10002,823
1,10080,291
2,10120,192
3,10125,1225
4,10133,2363


In [15]:
# Total Revenue per Product

data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

revenue_per_product = data.groupby('StockCode')['TotalPrice'].sum().reset_index().round(2)
revenue_per_product.head(5)

,StockCode,TotalPrice
0,10002,699.55
1,10080,114.41
2,10120,40.32
3,10125,929.45
4,10133,1134.79


In [16]:
# Total Revenue by Country

revenue_by_country = data.groupby('Country')['TotalPrice'].sum().reset_index().round(2)
revenue_by_country.head(5)

,Country,TotalPrice
0,Australia,137009.77
1,Austria,10154.32
2,Bahrain,548.40
3,Belgium,40910.96
4,Brazil,1143.60


*Reshaping Data*

In [17]:
data['Month'] = data['InvoiceDate'].dt.to_period('M')
monthly_sales = data.pivot_table(values = 'TotalPrice', index = 'Month', aggfunc = 'sum').round(2)
monthly_sales

,TotalPrice
Month,
2010-12,552372.86
2011-01,473731.90
2011-02,435534.07
2011-03,578576.21
2011-04,425222.67
2011-05,647011.67
2011-06,606862.52
2011-07,573112.32
2011-08,615078.09


In [18]:
customer_spending = data.pivot_table(values='TotalPrice', index='CustomerID', aggfunc='sum')
customer_spending.head()

,TotalPrice
CustomerID,
12346,0.00
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


## Key Metrics

*1. Transaction & Revenue Metrics*

- *Total Revenue = Sum(Quantity × UnitPrice)*

In [19]:
total_revenue = data['TotalPrice'].sum().round(2)

- *Total Transactions = Count of unique InvoiceNo*

In [20]:
total_transaction = data['InvoiceNo'].nunique()
total_transaction

22190

- *Average Order Value (AOV) = Total Revenue / Total Transactions*

In [21]:
average_order_value = (total_revenue / total_transaction) * 100
average_order_value.round(2)

np.float64(37307.43)

- *Monthly Revenue Trend = Revenue aggregated per InvoiceDate*

In [22]:
data['Month'] = data['InvoiceDate'].dt.to_period('M')
monthly_sales = data.pivot_table(values = 'TotalPrice', index = 'Month', aggfunc = 'sum').round(2)
monthly_sales.head()

,TotalPrice
Month,
2010-12,552372.86
2011-01,473731.90
2011-02,435534.07
2011-03,578576.21
2011-04,425222.67


- *Top Revenue-Generating Invoices = Highest revenue per InvoiceNo*

In [23]:
invoivce_revenue = data.groupby('InvoiceNo')['TotalPrice']
top_invoices = data.pivot_table(values = 'TotalPrice', index = 'InvoiceNo').round(2)
top_invoices.sort_values(by = 'TotalPrice', ascending = False).head()

,TotalPrice
InvoiceNo,
581483,168469.60
541431,77183.60
556444,38970.00
551697,8142.75
540815,5053.63


*2. Product Performance Metrics*

- *Top-Selling Products = Products with the highest total Quantity sold*

In [24]:
top_selling_goods = data.groupby('Description')['Quantity'].sum()
top_selling_goods.sort_values( ascending = False).head(10)

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53119
JUMBO BAG RED RETROSPOT               44963
ASSORTED COLOUR BIRD ORNAMENT         35215
WHITE HANGING HEART T-LIGHT HOLDER    34128
PACK OF 72 RETROSPOT CAKE CASES       33386
POPCORN HOLDER                        30492
RABBIT NIGHT LIGHT                    27045
MINI PAINT SET VINTAGE                25880
PACK OF 12 LONDON TISSUES             25305
PACK OF 60 PINK PAISLEY CAKE CASES    24129
Name: Quantity, dtype: int64

- *Most Profitable Products = Products contributing the most to Total Revenue*

In [25]:
profitable_selling_goods = data.groupby('Description')['TotalPrice'].sum().round()
profitable_selling_goods.sort_values(ascending = False).head(10)

Description
REGENCY CAKESTAND 3 TIER              132568.0
WHITE HANGING HEART T-LIGHT HOLDER     93768.0
JUMBO BAG RED RETROSPOT                83057.0
PARTY BUNTING                          67628.0
POSTAGE                                66710.0
ASSORTED COLOUR BIRD ORNAMENT          56332.0
RABBIT NIGHT LIGHT                     51043.0
CHILLI LIGHTS                          45915.0
PAPER CHAIN KIT 50'S CHRISTMAS         41424.0
PICNIC BASKET WICKER 60 PIECES         39620.0
Name: TotalPrice, dtype: float64

- *Average Product Price = Mean of UnitPrice across all transactions*

In [26]:
data['UnitPrice'].mean().round(2)

np.float64(3.47)

- *Product Demand by Season = Total Quantity sold per quarter based on InvoiceDate*

In [27]:
data['Quarter'] = data['InvoiceDate'].dt.to_period('Q')
product_demand_season = data.groupby('Quarter')['Quantity'].sum().reset_index()
product_demand_season.sort_values(by = 'Quarter')

,Quarter,Quantity
0,2010Q4,295177
1,2011Q1,874093
2,2011Q2,1001084
3,2011Q3,1283574
4,2011Q4,1438923


- *Slow-Moving Products = Products with low sales volume over time*

In [28]:
slow_moving_goods = profitable_selling_goods.sort_values(ascending= True)
slow_moving_goods.head()

Description
Manual                         -58745.0
CRUK Commission                 -7933.0
Discount                        -5696.0
CHALKBOARD KITCHEN ORGANISER      -88.0
WHITE CHERRY LIGHTS               -54.0
Name: TotalPrice, dtype: float64

*3.  Customer Purchase Behavior*

- *Unique Customers = Count of unique CustomerID*

In [29]:
unique_customers = data['CustomerID'].nunique()
unique_customers

4372

- *Repeat Purchase Rate = Customers with multiple InvoiceNo transactions / Total Customers*

In [43]:
total_customers = data['CustomerID'].count()
repeat_customers = data.groupby('CustomerID')['InvoiceNo'].nunique()
repeat_purchase_rate =( repeat_customers / total_customers ) * 100
repeat_purchase_rate.head()

CustomerID
12346    0.000498
12347    0.001743
12348    0.000996
12349    0.000249
12350    0.000249
Name: InvoiceNo, dtype: float64

- *Average Purchase Frequency = Total Transactions / Unique Customers*

In [48]:
average_purchase_frequency = ( total_transaction / unique_customers ) * 100
print(round(average_purchase_frequency, 2))

507.55


- *High-Value Customers = Customers contributing the highest Total Revenue*

In [52]:
high_value_customers = data.groupby('CustomerID')['TotalPrice'].sum()
high_value_customers.sort_values(ascending=False).head(10)

CustomerID
14646    279489.02
18102    256438.49
17450    187322.17
14911    132458.73
12415    123725.45
14156    113214.59
17511     88125.38
16684     65892.08
13694     62690.54
15311     59284.19
Name: TotalPrice, dtype: float64

- *Customer Segmentation by Spending = Grouping customers based on Total Spend per CustomerID*

In [54]:
high_value_customers.sort_values(ascending=False).head(20)

CustomerID
14646    279489.02
18102    256438.49
17450    187322.17
14911    132458.73
12415    123725.45
14156    113214.59
17511     88125.38
16684     65892.08
13694     62690.54
15311     59284.19
13089     57322.13
14096     57120.91
15061     54228.74
16029     53168.69
17949     52750.84
15769     51823.72
14298     50862.44
14088     50415.49
17841     39869.05
13798     36352.87
Name: TotalPrice, dtype: float64

*4. Geographical Sales Metrics*

- *Total Sales by Country = Revenue aggregated by Country*

In [61]:
sales_by_country = data.groupby('Country')['TotalPrice'].sum().round()
sales_by_country

Country
Australia                137010.0
Austria                   10154.0
Bahrain                     548.0
Belgium                   40911.0
Brazil                     1144.0
Canada                     3666.0
Channel Islands           20076.0
Cyprus                    12859.0
Czech Republic              708.0
Denmark                   18768.0
EIRE                     250002.0
European Community         1292.0
Finland                   22327.0
France                   196626.0
Germany                  221509.0
Greece                     4711.0
Iceland                    4310.0
Israel                     6988.0
Italy                     16891.0
Japan                     35341.0
Lebanon                    1694.0
Lithuania                  1661.0
Malta                      2505.0
Netherlands              284662.0
Norway                    35163.0
Poland                     7213.0
Portugal                  28996.0
RSA                        1002.0
Saudi Arabia                131.0
Singap

- *Top Revenue-Generating Countries = Highest Total Revenue per country*

In [69]:
revenue_per_country = data.groupby('Country')['TotalPrice'].sum()
average_price = revenue_per_country.mean().round()
top_revenue_country = revenue_per_country[revenue_per_country > average_price]
top_revenue_country.round(2)

Country
EIRE               250001.78
Netherlands        284661.54
United Kingdom    6747156.15
Name: TotalPrice, dtype: float64

- *Regional Product Preferences = Most sold StockCode per Country*

In [73]:
regional_product_preferences = data.groupby('Country')['StockCode'].count()
regional_product_preferences.head(10)

Country
Australia          1258
Austria             401
Bahrain              17
Belgium            2069
Brazil               32
Canada              151
Channel Islands     757
Cyprus              611
Czech Republic       30
Denmark             389
Name: StockCode, dtype: int64

- *Country-Wise Average Order Value = Average AOV per Country*

In [76]:
country_average_order_value = sales_by_country.mean()
country_average_order_value.round(2)

np.float64(223743.73)

*5. Invnetory & Stock efficiecy (Total Inventory is not avialable)*

- *Stock Turnover Rate = (Total Quantity Sold) / (Total Inventory Available)*

In [ ]:
total_quantity = data.groupby('StockCode')['Quantity'].sum()
total_inventory = data['Quantity'].sum()
stock_turnover_rate =  (total_quantity / total_inventory) / 2
stock_turnover_rate.round(2).head()

StockCode
10002    0.0
10080    0.0
10120    0.0
10125    0.0
10133    0.0
Name: Quantity, dtype: float64

- *Overstocked Products = Products with high StockCode availability but low Quantity Sold*

- *Demand Forecasting = Predictive modeling based on past Quantity Sold trends*